# Introduction to Dimensionality Reduction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA
from scipy.stats import zscore
import pandas as pd
from itertools import combinations
from mpl_toolkits.axes_grid1 import make_axes_locatable
from seaborn import lmplot
import pickle
import random
import scanpy as sc
import anndata
from IPython.display import display, clear_output

#### A walk on the beach

<center><img src="beach.jpeg" width="600"></center>
<div style="text-align: center"> source: https://www.publicdomainpictures.net/en/free-download.php?image=shadows-on-the-beach&id=177457 </div>

#### Looking for repetition: a recipe book

<center><img src="recipes2.png" width="800"></center>

## Principal Components Analysis (PCA)

With PCA, data from a high-dimensional space (e.g. 2D, a plane) can be projected onto a lower-dimensional space (e.g. 1D, a line).

<center><img src="pca_proj.jpeg" width="500"></center>
<div style="text-align: center"> source: https://programmathically.com/principal-components-analysis-explained-for-dummies/ </div>

Here is an example of 3D data projected onto a 2D plane.

<center><img src="pca.png" width="800"></center>
<div style="text-align: center"> source: https://www.publicdomainpictures.net/en/free-download.php?image=shadows-on-the-beach&id=177457 </div>

In [ ]:
## Import iris dataset
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
## Store data in pandas DataFrame
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

## Rename columns
iris_df['target'] = iris_df['target'].map({0:iris.target_names[0], 1:iris.target_names[1], 2:iris.target_names[2]})
iris_df.rename(columns = {'target':'species'}, inplace=True)

## Display data and list species' names
display(iris_df)
print(iris_df['species'].unique())

In [ ]:
## Extract numerical values in arrays
x = iris_df.iloc[:,:-1].values
x

In [ ]:
## Normalize data: zero mean & unit variance

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)
x_trunc = x[:,:-1]  ## first three features, for visualization

In [ ]:
## Compare histograms of features before and after applying the standard scaler

print('Before scaling')
iris_df.hist(sharex=True); plt.show()

print('After scaling')
pd.DataFrame(x, columns=iris_df.columns[:-1].str.strip(' (cm)')).hist(sharex=True); plt.show()

In [ ]:
## Visualize truncated data containing first three features

fig = plt.figure(1, figsize=(12, 8))
ax = fig.add_subplot(111, projection="3d", elev=-150, azim=110)

ax.scatter(x_trunc[:,0], x_trunc[:,1], x_trunc[:,2]);

ax.set_xlabel(iris_df.columns[0]); ax.set_ylabel(iris_df.columns[1]); ax.set_zlabel(iris_df.columns[2])
plt.show()

In [ ]:
## Compare 2D projections of full vs truncated data

from sklearn.decomposition import PCA

fig, ax = plt.subplots(1,1,figsize=[6.,6.])
ax.set_xlabel('PCA 1'); ax.set_ylabel('PCA 2')
ax.set_xticks([]); ax.set_yticks([])

## 2D PCA projection of truncated data
pca = PCA(n_components=2)
x_2d = pca.fit_transform(x_trunc)
ax.scatter(x_2d[:,0], x_2d[:,1], label='truncated data\n(3 features)')

## 2D PCA projection of full data
pca = PCA(n_components=2)
x_2d = pca.fit_transform(x)
ax.scatter(x_2d[:,0], x_2d[:,1], label='full data\n(4 features)')

plt.legend()
plt.show()

In [ ]:
## Show "manual" 2D PCA projection alongside PCA weights for different features

fig, ax = plt.subplots(1,2,figsize=[12,6])
ax[0].set_xlabel('PCA 1')  # proportion of variance explained by PCA component 1
ax[0].set_ylabel('PCA 2')  # proportion of variance explained by PCA component 2
ax[0].set_xticks([]); ax[0].set_yticks([])
ax[0].set_title('PCA projection ($\hat{X} = XU^T$)')

## Matrix multiplication to obtain projection
x_2d_manual = x @ pca.components_[:2,:].T
ax[0].scatter(x_2d_manual[:,0], x_2d_manual[:,1]);

xlim = ax[0].get_xlim()
ylim = ax[0].get_ylim()

## Barplot of components' weights
bp = pd.DataFrame(pca.components_, columns=iris_df.columns[:-1].str.strip(' (cm)'), index=['PCA 1', 'PCA 2']).plot.bar(ax=ax[1], rot=0);

bp.set_ylabel('weight');
for p in ax[1].patches: ax[1].annotate(str(round(p.get_height(), 2)), (p.get_x()-0.005, p.get_height()*1.025))
ax[1].set_title('PCA weights ($U$)')

plt.tight_layout()
plt.show()

$$
PCA.1 = 0.52 * sepal.length - 0.27 * sepal.width + 0.58 * petal.length + 0.56 * petal.width\\
PCA.2 = 0.38 * sepal.length - 0.92 * sepal.width + 0.02 * petal.length + 0.07 * petal.width
$$

Performing a PCA projection amounts to applying the equations above to create PCA variables, which we can plot. In the code below, we will loop through each data point to compute PCA projections using the data features and PCA weights.

In [ ]:
## Create figure

fig = plt.figure(figsize=[8,8])
ax = fig.add_subplot(1, 1, 1) 
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')


## Loop through data points
for i in range(len(x)):
    
    xi = x[i,:].squeeze()
    
    c1 = pca.components_[0,:]
    c2 = pca.components_[1,:]
    
    xi_pc1 = (xi * c1).sum()
    xi_pc2 = (xi * c2).sum()
    ax.scatter(xi_pc1, xi_pc2, c='blue')
    
    dpi = 'Data point ' + str(i)
    pc1_1 = '({:.2f})*({:.2f})'.format(c1[0] , xi[0])
    pc1_2 = '({:.2f})*({:.2f})'.format(c1[1] , xi[1])
    pc1_3 = '({:.2f})*({:.2f})'.format(c1[2] , xi[2])
    pc1_4 = '({:.2f})*({:.2f})'.format(c1[3] , xi[3])

    pc2_1 = '({:.2f})*({:.2f})'.format(c2[0] , xi[0])
    pc2_2 = '({:.2f})*({:.2f})'.format(c2[1] , xi[1])
    pc2_3 = '({:.2f})*({:.2f})'.format(c2[2] , xi[2])
    pc2_4 = '({:.2f})*({:.2f})'.format(c2[3] , xi[3])

    pc_info = str(
    '''
    {:<16}: {:>15}    {:>15}    {:>15}    {:>15}
    {:<16}: {:>15.2f}    {:>15.2f}    {:>15.2f}    {:>15.2f}
    {:<16}: {:>16} + {:>16} + {:>16} + {:>16} = {:>5.2f}
    {:<16}: {:>16} + {:>16} + {:>16} + {:>16} = {:>5.2f}
    '''.format(
    'features', 'sepal length', 'sepal width', 'petal length', 'petal width',
    dpi, *xi,
    'PCA 1' , pc1_1 , pc1_2 , pc1_3 , pc1_4, xi_pc1,
    'PCA 2' , pc2_1 , pc2_2 , pc2_3 , pc2_4, xi_pc2))
    
    display(fig)
    print(pc_info)
  
    clear_output(wait = True)
    plt.pause(0.5)

In [ ]:
## "Manual" 2D projection - project dimensions individually for one data point

xi = x[0,:]
print('Feature values of xi:', xi, '\n')
print('Weights for PCA 1:', pca.components_[0,:])
print('Weights for PCA 2:', pca.components_[1,:], '\n')

xi_1 = (xi * pca.components_[0,:]).sum()  # project onto 1st component
xi_2 = (xi * pca.components_[1,:]).sum()  # project onto 2nd component

print('Projection of xi onto PCA 1 and PCA 2 - manual computation:', np.float32([xi_1, xi_2]))
print('Projection of xi onto PCA 1 and PCA 2 - automatic computation:', np.float32(x_2d[0,:]))

In [ ]:
## Compare full "manual" 2D projection to automatic 2D projection

fig, ax = plt.subplots(1,1,figsize=[6.,6.])
ax.set_xlabel('PCA 1'); ax.set_ylabel('PCA 2')
ax.set_xticks([]); ax.set_yticks([])

## 2D PCA projection of full data
pca = PCA(n_components=2)
x_2d = pca.fit_transform(x)
ax.scatter(x_2d[:,0], x_2d[:,1], label='automatic projection')

## 2D PCA projection of full data - matrix multiplication
x_2d_manual = x @ pca.components_.T
ax.scatter(x_2d_manual[:,0], x_2d_manual[:,1], label='manual projection');

plt.legend()
plt.show()

In [ ]:
## Color data points by species

iris_df_2d = pd.DataFrame(data = np.c_[x_2d, iris_df['species']], columns=['PCA 1', 'PCA 2', 'species'])
lmplot(x='PCA 1', y='PCA 2', data=iris_df_2d, hue='species', fit_reg=False);

In [ ]:
## Compute 4 PCA components rather than just 2 components
pca_4d = PCA(n_components=4)
x_4d = pca_4d.fit_transform(x)

## Plot all 6 possible 2D PCA projections
p = np.arange(4)
pcombs = list(combinations(p,2))

plt_cols = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors = {'setosa':plt_cols[0], 'versicolor':plt_cols[1], 'virginica':plt_cols[2]}

_, ax = plt.subplots(2,3, figsize=[15.,10.])
for i, pcomb in enumerate(pcombs):
    axi = np.unravel_index(i, ax.shape)
    ax[axi].scatter(x_4d[:,pcomb[0]], x_4d[:,pcomb[1]], c=iris_df['species'].map(colors))
    
    ax[axi].set_xlabel('PCA ' + str(pcomb[0]+1)); ax[axi].set_ylabel('PCA ' + str(pcomb[1]+1))
    ax[axi].set_xticks([]); ax[axi].set_yticks([])
    

## Plot singular values and related quantities
fig, ax = plt.subplots(1,4,figsize=[15.,3.])

sv_1 = pca_4d.singular_values_
sv_2 = pca_4d.singular_values_**2
sv_3 = pca_4d.singular_values_**2 / (pca_4d.singular_values_**2).sum()
sv_4 = pca_4d.explained_variance_ratio_

pd.DataFrame(sv_1, index=['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4']).plot.bar(ax=ax[0], rot=0, legend=False, color='cyan', title='singular values');
for p in ax[0].patches: ax[0].annotate(str(round(p.get_height(), 2)), (p.get_x()+0.01, p.get_height()*0.9))
pd.DataFrame(sv_2, index=['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4']).plot.bar(ax=ax[1], rot=0, legend=False, color='cyan', title='squared singular values');
for p in ax[1].patches: ax[1].annotate(str(round(p.get_height(), 2)), (p.get_x()+0.01, p.get_height()*0.9))
pd.DataFrame(sv_3, index=['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4']).plot.bar(ax=ax[2], rot=0, legend=False, color='cyan', title='ratio of squared sing. values');
for p in ax[2].patches: ax[2].annotate(str(round(p.get_height(), 2)), (p.get_x()+0.01, p.get_height()*0.9))
pd.DataFrame(sv_4, index=['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4']).plot.bar(ax=ax[3], rot=0, legend=False, color='cyan', title='variance explained');
for p in ax[3].patches: ax[3].annotate(str(round(p.get_height(), 2)), (p.get_x()+0.01, p.get_height()*0.9))

In [ ]:
## Plot cumulative sum of variances explained by the PCA components

_, ax = plt.subplots(1,1)
pd.DataFrame(sv_4.cumsum(), index=['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4']).plot.bar(ax=ax, rot=0, legend=False, color='cyan', title='variance explained');
for p in ax.patches: ax.annotate(str(round(p.get_height(), 2)), (p.get_x()+0.1, p.get_height()*0.9))

#### The biplot

We've plotted the projections of individual samples onto the 2D PCA represenation, effectively achieving dimensionality reduction for the purposes of e.g. data visualization. We've also *separately* plotted the weights of the PCA components to visualize how different features are weighted when projecting data onto the 2D representation.

Alternatively, one can visualize *both* the samples' projection onto the 2D representation as well as the features' weights simultaneously via a **biplot**.

In [ ]:
def biplot(score, coeff , y, variables, extend=1):
    '''
    
    ***** Author: Serafeim Loukas, serafeim.loukas@epfl.ch *****
    https://ai.plainenglish.io/how-to-implement-pca-with-python-and-scikit-learn-22f3de4e5983
    
    Inputs:
       score: the projected data
       coeff: the eigenvectors (PCs)
       y: the class labels
   '''
    
    xs = score[:,0] # projection on PC1
    ys = score[:,1] # projection on PC2
    n = coeff.shape[0] # number of variables
    plt.figure(figsize=(8,6), dpi=100)
    classes = np.unique(y)
    for s,l in enumerate(classes):
        plt.scatter(xs[y==l],ys[y==l], label=l) # color based on group
    plt.legend()
    for i in range(n):
        #plot as arrows the variable scores (each variable has a score for PC1 and one for PC2)
        plt.arrow(0, 0, coeff[i,0] * extend, coeff[i,1] * extend, color = 'k', alpha = 0.9,linestyle = '-',linewidth = 1.5, overhang=0.2)
        plt.text(coeff[i,0]* extend, coeff[i,1] * extend, variables[i], color = 'k', ha = 'center', va = 'center',fontsize=10)
    plt.xlabel("PCA {}".format(1), size=14)
    plt.ylabel("PCA {}".format(2), size=14)
    limx= int(xs.max()) + 1
    limy= int(ys.max()) + 1
    plt.xlim([-limx,limx])
    plt.ylim([-limy,limy])
    plt.grid()
    plt.tick_params(axis='both', which='both', labelsize=14)
    
    
def bisubplot(score, coeff , y, variables, pcomb, ax, extend=1):
    '''
    
    ***** Author: Serafeim Loukas, serafeim.loukas@epfl.ch *****
    https://ai.plainenglish.io/how-to-implement-pca-with-python-and-scikit-learn-22f3de4e5983
    
    Inputs:
       score: the projected data
       coeff: the eigenvectors (PCs)
       y: the class labels
   '''
    
    xs = score[:,0] # projection on PC1
    ys = score[:,1] # projection on PC2
    n = coeff.shape[0] # number of variables
    classes = np.unique(y)
    for s,l in enumerate(classes):
        ax.scatter(xs[y==l],ys[y==l], label=l) # color based on group
    ax.legend()
    for i in range(n):
        #plot as arrows the variable scores (each variable has a score for PC1 and one for PC2)
        ax.arrow(0, 0, coeff[i,0] * extend, coeff[i,1] * extend, color = 'k', alpha = 0.9,linestyle = '-',linewidth = 1.5, overhang=0.2)
        ax.text(coeff[i,0]* extend, coeff[i,1] * extend, variables[i], color = 'k', ha = 'center', va = 'center',fontsize=10)
    ax.set_xlabel("PCA {}".format(pcomb[0]+1), size=14)
    ax.set_ylabel("PCA {}".format(pcomb[1]+1), size=14)
    limx= int(xs.max()) + 1
    limy= int(ys.max()) + 1
    ax.set_xlim([-limx,limx])
    ax.set_ylim([-limy,limy])
    ax.grid()
    ax.tick_params(axis='both', which='both', labelsize=14)

In [ ]:
## Barplot of components' weights

fig, ax = plt.subplots(1,3,figsize=[14.,4.5])

ax[0].scatter(x_2d[:,0], x_2d[:,1], c=iris_df['species'].map(colors))
ax[0].set_xlabel('PCA 1'); ax[0].set_ylabel('PCA 2')

## PCA weights as plotted previously
bp = pd.DataFrame(pca.components_, columns=iris_df.columns[:-1].str.strip(' (cm)'), index=['PCA 1', 'PCA 2']).plot.bar(ax=ax[1], rot=0);

bp.set_ylabel('weights');
for p in ax[1].patches:
    ax[1].annotate(str(round(p.get_height(), 2)), (p.get_x()-0.005, p.get_height()*1.025))
    
## PCA weights, plotted by feature rather than PCA component
bp = pd.DataFrame(pca.components_.T, columns=['PCA 1','PCA 2'], index=iris_df.columns[:-1].str.strip(' (cm)')).plot.bar(ax=ax[2], rot=0, color=['pink','cyan']);

bp.set_ylabel('weights');
for p in ax[2].patches:
    ax[2].annotate(str(round(p.get_height(), 2)), (p.get_x()-0.005, p.get_height()*1.025))
    
plt.tight_layout()
plt.show()

In [ ]:
## Create biplot
biplot(x_2d, pca.components_.T, iris_df['species'], iris_df.columns[:-1].str.strip(' (cm)'), extend=1)

In [ ]:
## Histograms of original features, grouped by species
fig, ax = plt.subplots(1,4,figsize=[18,3])
for c,col in enumerate(iris_df.columns[:-1]):
    h = iris_df.pivot(columns='species', values=col).plot.hist(alpha=0.8, title=col, ax=ax[c])

plt.tight_layout()
plt.show()

In [ ]:
## Plot all 6 possible biplots arising from the 4 PCA components

_, ax = plt.subplots(3,2, figsize=[15.,24.])
for i, pcomb in enumerate(pcombs):
    axi = np.unravel_index(i, ax.shape)
    bisubplot(x_4d[:,pcomb], pca_4d.components_[pcomb,:].T, iris_df['species'], iris_df.columns[:-1].str.strip(' (cm)'), pcomb, ax[axi], extend=1)

#### Kernel PCA

PCA provides *linear* components which can be unsuitable for data with non-linear structures. Non-linear functions called *kernels* can be used to represent data such that PCA can provide suitable linear components of that non-linear representation. This method is termed **Kernel PCA**.

<img src="kpca.png" width="900">
<div style="text-align: center"> source: https://www.cs.mcgill.ca/~dprecup/courses/ML/Lectures/ml-lecture13.pdf </div>

#### Probabilistic PCA
PCA does not attempt to model noise that accompany its components. In **Probabilistic PCA**, a noise variance term $\sigma^2$ accounts for random variations within its components, allowing projections against its components with varying spreads. Probabilistic PCA can thus model both the linear structure and noise components of its components. Due to its probabilistic nature, Probabilistic PCA can also be used as a generative model. Note that Probabilistic PCA can be viewed as a special case of **Factor Analysis**.

<img src="ppca.png" width="1000">
<div style="text-align: center"> source: https://www.youtube.com/watch?v=lJ0cXPoEozg </div>

### Independent Component Analysis (ICA)

PCA provides *orthogonal* components / coordinates, which can limit the modelling capacities of PCA. On the other hand, such ICA components are not constrained to be orthogonal which provides ICA with more "flexibility".

<center><img src="pca_vs_ica.png" width="800"></center>
<div style="text-align: center"> source: https://brainvision.com/applications/eeg/ </div>

#### Electroencephalography (EEG) data analysis
 
 The EEG is a brain recording modality where electrodes are placed on the scalp/head to measure fluctuations in bioelectical potentials. Hence, every electrode measures a time-varying signal.
 
  <center><img src="eeg_image.png" width="400"></center>
  <div style="text-align: center"> source: https://brainvision.com/applications/eeg/ </div>

Import EEG data using [MNE Python](https://mne.tools/stable/index.html) analysis package. We will be using open-source data which has been reported in:

[Babayan, A., Erbey, M., Kumral, D. et al. *A mind-brain-body dataset of MRI, EEG, cognition, emotion, and peripheral physiology in young and old adults.* Sci Data 6, 180308 (2019).](https://www.nature.com/articles/sdata2018308)

In [ ]:
## Load EEG data
eeg = mne.io.read_raw_eeglab("sub-010321_EC_downsamp.set")
eeg.annotations.delete( np.arange( len(eeg.annotations.description) ) )  # remove annotations, not important

In [ ]:
## Plot EEG electrode layout and EEG signals
eeg.plot_sensors(show_names=True, sphere=90);
eeg.plot(n_channels=12, duration=4);  # recommend plotting using "notebook" plotting backend, i.e. %matplotlib notebook

From the time-series plots, it seems that many time-series are alike. Perhaps there's a way to find patterns and have a more succinct representation of our data. One way to find these patterns is to use **Independent Components Analysis** (ICA) on our EEG data.

  <center><img src="eeg_ica.jpeg" width="500"></center>
  <div style="text-align: center"> source: https://sccn.ucsd.edu/~jung/Site/EEG_artifact_removal.html </div>

In [ ]:
## Compute ICA on EEG data
num_comps = 15  # choose number of patterns, or "components"

ica = ICA(n_components=num_comps, random_state=97)
ica.fit(eeg)

In [ ]:
## Plot ICA components
ica.plot_sources(eeg, start=10, stop=16);
ica.plot_components();

In [ ]:
## Compute PCA on EEG data
num_comps = 15  # choose number of patterns, or "components"

eeg_ts = eeg.get_data().T
pca = PCA(n_components=num_comps)
pca_data_ts = pca.fit_transform(eeg_ts)

In [ ]:
## Plot PCA components
len_plot = 1000     # choose time-series length for plotting purposes

num_plots = pca_data_ts.shape[1]
plt.figure(figsize=[9.,9.])

for i in range(num_plots):
    ax = plt.subplot( num_plots , 1, i+1 )
    plt.plot(zscore(pca_data_ts)[:len_plot,i])
    
    ax.set_ylabel("PCA {}".format(i), fontsize=8, rotation=0)
    ax.set_xticks([]); ax.set_yticks([])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    
plt.tight_layout()
plt.show()

### Nonlinear dimensionality reduction: t-SNE and UMAP

Linear dimensionality reduction methods such as PCA may be incapable of properly displaying data in a low-dimensional space (e.g. 2D). As previously discussed, kernel PCA leverages nonlinear transformations (i.e. kernels) such that the resulting representation is ammenable to standard PCA. However, kernel PCA requires a pre-defined kernel function to obtain a suitable nonlinear representation.

We will now explore two nonlinear dimensionality reduction methods which do not require such pre-defined kernels. These are:
1. t-distributed Stochastic Neighbor Embedding (**t-SNE**)
2. Uniform Manifold Approximation and Projection (**UMAP**)

Essentially, t-SNE and UMAP aim to preserve the "similarities" between pairs of datapoints. In other words, points which are "similar" in high-dimensional space should remain "similar" in low-dimensional space, where similarities are quantified kernels computed over distances. The concept of a *graph* is thus relevant for t-SNE and UMAP.

<center><img src="umap.png" width="800"></center>
.
<div style="text-align: center"> source: https://umap-learn.readthedocs.io/en/latest/parametric_umap.html </div>

Although UMAP and t-SNE are very similar methods for embedding high-dimensional data into low-dimensional space, they have their differences. These methods use different kernels to quantify distances, symmetrize distances differently, have different number of parameters, and initialize low-dimensional embeddings differently. To the last point, t-SNE initializes low-dimensional embeddings randomly, whereas UMAP does so using spectral embedding. UMAP should thus provide much more stable and reproducible results compared to t-SNE.

Another (debated) difference between t-SNE and UMAP is the ability to preserve *global* structure in addition to *local* structure. One way of understanding this distinction is to view the graph structure in low-dimensional space as a set of attractive and repulsive forces between datapoints. The low-dimensional embedding is obtained once the forces equilibrate, similar to a set of springs left at its minimum energy equilibrium.

<center><img src="spring_graph.png" width="400"></center>
<div style="text-align: center"> source: Dr Laleh Haghverdi - sc-sys-med Summer School (31 May, 2022) </div>

#### Warning: speculative
In t-SNE, the balance between attractive and repulsive forces are achieved via the minimization of the Kullback-Leibler divergence, whereas minimization in UMAP is achieved via cross-entropy. The latter directly accounts for repulsive forces between dis-similar points whereas the Kullback-Leibler divergence does not. This difference between cross-entropy (UMAP) and Kullback-Leibler divergence (t-SNE) perhaps accounts for the better preservation of global structure in UMAP, although other theoretically-grounded reasons are also at play.

**Note**: t-SNE and UMAP are primarily used as *data visualization* methods and should generally not be used for other downstream purposes usually associated with dimensionality reduction such as with PCA.

**Note**: Clustering is often performed separately using e.g. the Louvain or Leiden algorithm. Datapoints embedded within the low-dimensional t-SNE/UMAP space are then colour-coded by cluster label.

#### t-SNE and UMAP on scRNA-seq data
t-SNE and UMAP are often used in the analysis of single-cell RNA sequencing (scRNA-seq) data which is very high dimensional, e.g. d = 20'000 genes. scRNA-seq data from the mouse prancreas will be used here for demonstration.

In [ ]:
## Import and format data

random.seed(10)

# mouse pancreas single-cell dataset
# read in data and cell type labels
with open('MP.pickle', 'rb') as f:
    df = pickle.load(f)

with open('MP_genes.pickle', 'rb') as f:
    genes = pickle.load(f)

df.set_index('Unnamed: 0', inplace=True)  # set first column (cell ID as the index column)
sample_id = pickle.load(open('cell_IDs.pkl', 'rb'))
df = df.loc[list(sample_id), :]

X = df[genes].values  # extract the N x M cells-by-genes matrix

sample_info = pd.read_csv('sample_info.csv')

mp_anndata = anndata.AnnData(X=X)
mp_anndata.obs['Celltype'] = sample_info['assigned_cluster'].values
celltype_one_hot = pd.get_dummies(mp_anndata.obs['Celltype'], prefix='Celltype').values

N = X.shape[0]  # number of single-cell samples
K = len(sample_info['assigned_cluster'].unique())  # number of topics
M = X.shape[1]  # number of genes

display(mp_anndata)

#### PCA

In [ ]:
## Compute and plot 2D PCA projection
_, ax = plt.subplots(figsize=(7, 6))

sc.tl.pca(mp_anndata)
sc.pl.pca(mp_anndata, color=["Celltype"], ax=ax, title='two-dimensional PCA plot')

#### t-SNE

In [ ]:
## Compute and plot 2D t-SNE projection
_, ax = plt.subplots(figsize=(7, 6))

sc.tl.tsne(mp_anndata, perplexity=30)
sc.pl.tsne(mp_anndata, color=["Celltype"], ax=ax, title='two-dimensional tSNE plot')

#### The perplexity parameter in t-SNE

The *perplexity* parameter modulates the width of the Gaussian kernel used to quantify similarities between points based on their distance of separation. Larger perplexity values lead to wider Gaussian kernels, allowing greater similarity between points that are more distant from one another.

<center><img src="tsne_perplexity.png" width="800"></center>
<div style="text-align: center"> source: Dr Laleh Haghverdi - sc-sys-med Summer School (31 May, 2022) </div>

#### UMAP

In [ ]:
## Compute neighbor affinities
sc.pp.neighbors(mp_anndata, n_neighbors=30)

## Compute and plot 2D UMAP projection
_, ax = plt.subplots(figsize=(7, 6))

sc.tl.umap(mp_anndata, min_dist=0.8)
sc.pl.umap(mp_anndata, color=["Celltype"], ax=ax, title='two-dimensional UMAP plot')

#### Number of nearest neighbors and Minimum distance parameters in UMAP

UMAP relies on the initial construction of a high-dimensional graph which is then embedded in low-dimensional space. This high-dimensional graph relies on the **number of nearest neighbors** parameter (n_neighbors) which controls how many adjacent datapoints are used to construct adaptive similarity kernels. To estimate the low-dimensional embedding, UMAP relies on the **minimum distance** parameter to "re-calibrate" the similarities' sensitivity to more distant points.

In short, both parameters aim to balance local and global structures: *n_neighbors* does so during the initial contruction of the high-dimensional graph, whereas *min_distance* does so later during the search for a low-dimensional embedding. As such, altering *n_neighbors* has a more severe impact than *min_distance*.

<center><img src="umap_n_neighbors_min_dist.png" width="1000"></center>
<div style="text-align: center"> source: https://pair-code.github.io/understanding-umap/ </div>

**Note**: other parameters are at play for UMAP, but are considered less interpretable

#### Recommended reading and exercise:
- Understanding UMAP (and t-SNE): https://pair-code.github.io/understanding-umap/

- Fine-tuning UMAP visualizations: https://jlmelville.github.io/uwot/abparams.html

## Autoencoders

#### A word about neural networks
Neural networks combine linear and non-linear transformations to obtain powerful hidden representations of data. These hidden representations serve many purposes such as regression, classification, probability density estimation, image segmentation, etc.

<center><img src="neuralnet.png" width="400"></center>
<div style="text-align: center"> source: https://en.wikipedia.org/wiki/Neural_network </div>

#### Autoencoders

Within the context of dimensionality reduction, autoencoders are a subclass of neural networks which contains a hidden layer whose size is *smaller* than that of the input layer. Moreover, the output layer of an autoencoder aims at *reconstructing* the data provided at the input layer.

<center><img src="autoencoder.png" width="700"></center>
<div style="text-align: center"> source: https://www.jeremyjordan.me/autoencoders/ </div>

One can analyse the learned variables contained within the hidden layer. In this way, we achieve dimensionality reduction since these hidden variables are smaller in number than the original input variables.

#### Variational Autoencoders

We've briefly discussed how PCA can be framed within a probabilistic setting, leading to Probabilistic PCA. Similarly, **variational autoencoders** (VAE) frame the autoencoder framework within a probabilistic setting. Rather than estimate hidden variables within the bottleneck layer, VAEs estimate *hidden probability distributions* from which hidden variables can be sampled from.

<center><img src="vae.png" width="700"></center>
<div style="text-align: center"> source: https://www.jeremyjordan.me/autoencoders/ </div>

These hidden probability distributions are often chosen to follow the Gaussian/Normal distribution. This design provides structure to the bottleneck layer while also accounting for variability which is inherent within the data. Defining probability distributions within the bottleneck layer also has interesting implications for data generation.

In [ ]:
import keras
from keras import layers
from keras.datasets import mnist

**Note**: exercises adapted from: https://www.theaidream.com/post/an-introduction-to-autoencoder-and-variational-autoencoder-vae

#### MNIST data

<center><img src="mnist.jpeg" width="400"></center>
<div style="text-align: center"> source: https://github.com/cazala/mnist </div>

#### Construct and train autoencoder model

**Note**: There are three hidden layers in this model, where the middle bottleneck layer is called the *encoding* layer

In [ ]:
# This is the size of our encoded representations
encoding_dim = 2

hidden_dim = 64

# This is our input image
input_img = keras.Input(shape=(784,))

# "encoded" is the encoded representation of the input
hidden_enc = layers.Dense(hidden_dim, activation='relu')(input_img)
encoded = layers.Dense(encoding_dim, activation='relu')(hidden_enc)
ae_encoder = keras.Model(input_img, encoded, name='encoder')

# "decoded" is the lossy reconstruction of the input
encoded_inputs = keras.Input(shape=(encoding_dim,), name='z_sampling')
hidden_dec = layers.Dense(hidden_dim, activation='relu')(encoded_inputs)
decoded = layers.Dense(784, activation='sigmoid')(hidden_dec)
ae_decoder = keras.Model(encoded_inputs, decoded, name='decoder')

output_img = ae_decoder(ae_encoder(input_img))

autoencoder = keras.Model(input_img, output_img, name='ae')

#Now let's train our autoencoder to reconstruct MNIST digits.
#First, we'll configure our model to use a per-pixel binary crossentropy loss, and the Adam optimizer:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

#Let's prepare our input data. We're using MNIST digits, and we're discarding the labels (since we're only interested in encoding/decoding the input images).
(x_train, _), (x_test, y_test) = mnist.load_data()

#We will normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784.
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

#Now let's train our autoencoder for 50 epochs:
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test))


In [ ]:
## Encode and decode some digits

n = 10  # Number of digits to display

encoded_imgs = ae_encoder.predict(x_test, verbose=0)
decoded_imgs = ae_decoder.predict(encoded_imgs, verbose=0)

plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
## Plot MNIST samples in bottleneck layer of autoencoder model

x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

x_test_encoded = ae_encoder.predict(x_test, verbose=0)

plt.figure(figsize=(7, 6))
plt.scatter(x_test_encoded[:,0], x_test_encoded[:,1], c=y_test, cmap='tab10')
plt.xlabel('autoencoder 1')
plt.ylabel('autoencoder 2')
plt.colorbar()
plt.show()

In [ ]:
## Display a 2D manifold of the digits

n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))

# We will sample n points
grid_x = np.flip(np.linspace(0, 120, n))
grid_y = np.linspace(0, 100, n)

# Apply AE decoder along grid pattern
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = ae_decoder.predict(z_sample, verbose=0)  # decoder
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit
        
fig, ax = plt.subplots(1,1,figsize=(10, 10))
ax.imshow(figure)

ax.set_xticks( ticks=ax.get_xticks()[1:-1] , labels=np.linspace( grid_x[-1] , grid_x[0] , len(ax.get_xticks()[1:-1]) ))
ax.set_yticks( ticks=ax.get_yticks()[1:-1] , labels=np.linspace( grid_y[-1] , grid_y[0] , len(ax.get_yticks()[1:-1]) ))

ax.set_xlabel('autoencoder 1')
ax.set_ylabel('autoencoder 2')
plt.show()

#### Construct and train variational autoencoder model

In [ ]:
#First, here's our encoder network, mapping inputs to our latent distribution parameters:

latent_dim = 2

original_dim = 28 * 28
intermediate_dim = 64

inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

#We can use these parameters to sample new similar points from the latent space:
from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

#Finally, we can map these sampled latent points back to reconstructed inputs:
# Create encoder
vae_encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)

vae_decoder = keras.Model(latent_inputs, outputs, name='decoder')

# Instantiate VAE model
outputs = vae_decoder(vae_encoder(inputs)[2])

vae = keras.Model(inputs, outputs, name='vae_mlp')

#We train the model using the end-to-end model, with a custom loss function: the sum of a reconstruction term, and the KL divergence regularization term.

reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

vae.compile(optimizer='adam')

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))


#We train our VAE on MNIST digits:
vae.fit(x_train, x_train,
        epochs=10,
        batch_size=32,
        validation_data=(x_test, x_test))

In [ ]:
## Encode and decode some digits

x_test_encoded = vae_encoder.predict(x_test, verbose=0)
x_test_encoded = x_test_encoded[0]

plt.figure(figsize=(7, 6))
plt.scatter(x_test_encoded[:,0], x_test_encoded[:,1], c=y_test, cmap='tab10')
plt.xlabel('variational autoencoder 1')
plt.ylabel('variational autoencoder 2')
plt.colorbar()
plt.show()

In [ ]:
## Display a 2D manifold of the digits

n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))

# We will sample n points within [-15, 15]
grid_x = np.flip(np.linspace(-3, 3, n))
grid_y = np.linspace(-3, 3, n)

# Apply VAE decoder along grid pattern
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = vae_decoder.predict(z_sample, verbose=0)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

fig, ax = plt.subplots(1,1,figsize=(10, 10))
ax.imshow(figure)

ax.set_xticks( ticks=ax.get_xticks()[1:-1] , labels=np.linspace( grid_x[-1] , grid_x[0] , len(ax.get_xticks()[1:-1]) ))
ax.set_yticks( ticks=ax.get_yticks()[1:-1] , labels=np.linspace( grid_y[-1] , grid_y[0] , len(ax.get_yticks()[1:-1]) ))

ax.set_xlabel('variational autoencoder 1')
ax.set_ylabel('variational autoencoder 2')
plt.show()

#### What if the hidden layer of an autoencoder contains more than two hidden layers?

### Other topics in dimensionality reduction

1. Dimensionality reduction via latent associations between datasets
2. Tensor decompositions
3. Dimensionality reduction of autocorrelated data
4. Categorical and mixed data

#### Latent associations between two datasets

<center><img src="cca.png" width="400"></center>
<div style="text-align: center"> source: https://www.slideshare.net/blazf/canonical-correlation-analysis-319668 </div>

#### Tensor decomposition

<center><img src="tensor.png" width="800"></center>
<div style="text-align: center"> source: Hore et al. (2016) - Nature genetics </div>

#### Autocorrelated data

Many dimensionality reduction methods implicitly assume that the ordering of features is unimportant. However, this is not the case for autocorrelated signals such as EEG data where the ordering between features matters. Some methods such as Second-Order Blind Identification (SOBI) takes this ordering into account.

#### Categorical data
<center><img src="mca.jpeg" width="600"></center>
<div style="text-align: center"> source: Bannister (2019) - PLoS ONE </div>


For mixed data, where both numerical and categorical variables are present, one can use **Factor Analysis of Mixed Data** which combines PCA with Multiple Correspondance Analysis.